# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [102]:
import pandas as pd


columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
          'o', 'p']

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data", 
                 header=None, names=columns)

# expected observations = 690 instances 
df.shape

(690, 16)

In [103]:
pd.options.mode.use_inf_as_na = True # consider inf/-inf as na
df.isna().sum() # returns 0


df.head() 
#hmm, it says some data is missing, so let's filter for non numeric values


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [104]:


print(df[pd.to_numeric(df['b'], errors='coerce').isnull()]) 
# im choosing to drop all the ? rows because of the ambiguity of the data


df = df[df.b != '?']

     a  b       c  d  e   f   g      h  i  j  k  l  m      n     o  p
83   a  ?   3.500  u  g   d   v  3.000  t  f  0  t  g  00300     0  -
86   b  ?   0.375  u  g   d   v  0.875  t  f  0  t  s  00928     0  -
92   b  ?   5.000  y  p  aa   v  8.500  t  f  0  f  g  00000     0  -
97   b  ?   0.500  u  g   c  bb  0.835  t  f  0  t  s  00320     0  -
254  b  ?   0.625  u  g   k   v  0.250  f  f  0  f  g  00380  2010  -
286  a  ?   1.500  u  g  ff  ff  0.000  f  t  2  t  g  00200   105  -
329  b  ?   4.000  y  p   i   v  0.085  f  f  0  t  g  00411     0  -
445  a  ?  11.250  u  g  ff  ff  0.000  f  f  0  f  g      ?  5200  -
450  b  ?   3.000  y  p   i  bb  7.000  f  f  0  f  g  00000     1  -
500  b  ?   4.000  u  g   x   v  5.000  t  t  3  t  g  00290  2279  +
515  b  ?  10.500  u  g   x   v  6.500  t  f  0  f  g  00000     0  +
608  b  ?   0.040  y  p   d   v  4.250  f  f  0  t  g  00460     0  -


In [105]:
print(df[pd.to_numeric(df['b'], errors='coerce').isnull()]) #cool

Empty DataFrame
Columns: [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p]
Index: []


In [106]:
print(df[pd.to_numeric(df['n'], errors='coerce').isnull()])

df = df[df.n != '?']

     a      b       c  d  e   f   g       h  i  j  k  l  m  n      o  p
71   b  34.83   4.000  u  g   d  bb  12.500  t  f  0  t  g  ?      0  -
202  b  24.83   2.750  u  g   c   v   2.250  t  t  6  f  g  ?    600  +
206  a  71.58   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  +
243  a  18.75   7.500  u  g   q   v   2.710  t  t  5  f  g  ?  26726  +
270  b  37.58   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  +
278  b  24.58  13.500  y  p  ff  ff   0.000  f  f  0  f  g  ?      0  -
330  b  20.42   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  -
406  a  40.33   8.125  y  p   k   v   0.165  f  t  2  f  g  ?     18  -
456  b  34.58   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  -
592  b  23.17   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  +
622  a  25.58   0.000  ?  ?   ?   ?   0.000  f  f  0  f  p  ?      0  +
626  b  22.00   7.835  y  p   i  bb   0.165  f  f  0  t  g  ?      0  -


In [107]:
print(df[pd.to_numeric(df['n'], errors='coerce').isnull()])


df.shape # 666 :O pick up sticks

Empty DataFrame
Columns: [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p]
Index: []


(666, 16)

In [0]:
# change columns k, n, and o to float


df[['k', 'n', 'o']] = df[['k', 'n', 'o']].astype(dtype = float)

In [299]:
df.sort_values(['b'], ascending=False)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
539,b,80.25,5.500,u,g,?,?,0.540,t,f,0.0,f,g,0.0,340.0,-
550,b,76.75,22.290,u,g,e,z,12.750,t,t,1.0,t,g,0.0,109.0,+
485,b,74.83,19.000,y,p,ff,ff,0.040,f,t,2.0,f,g,0.0,351.0,-
585,b,73.42,17.750,u,g,ff,ff,0.000,t,f,0.0,t,g,0.0,0.0,+
405,a,69.50,6.000,u,g,ff,ff,0.000,f,f,0.0,f,s,0.0,0.0,-
296,b,69.17,9.000,u,g,ff,ff,4.000,f,t,1.0,f,g,70.0,6.0,-
157,a,68.67,15.000,u,g,e,z,0.000,t,t,14.0,f,g,0.0,3376.0,+
130,b,67.75,5.500,u,g,e,z,13.000,t,t,1.0,t,g,0.0,0.0,+
221,b,65.42,11.000,u,g,e,z,20.000,t,t,7.0,t,g,22.0,0.0,+
573,a,65.17,14.000,u,g,ff,ff,0.000,t,t,11.0,t,g,0.0,1400.0,+


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [149]:
from matplotlib import pyplot as plt

df.groupby('p').sum()








,c,h,k,n,o
p,,,,,
+,1788.07,1035.905,1400.0,49201.0,596325.0
-,1407.45,444.160,238.0,72088.0,68732.0


In [0]:

''' Relative relationship between variable c, skewed towards positive with h
    skewed heavily towards + with k, skewed towards - with n, MASSIVE skew + 
    with variable o'''

In [0]:
import numpy as np
from scipy import stats

In [165]:
contin_mean = df.groupby('p').mean()

contin_mean = contin_mean.T

contin_mean

p,+,-
c,5.980167,3.835014
h,3.464565,1.210245
k,4.682274,0.648501
n,164.551839,196.425068
o,1994.397993,187.280654


In [171]:
df.shape

(666, 16)

In [208]:
sample_size = 666
x , y = 0, 0

for row in contin_mean.iterrows():
  
  
  
  var_a = contin_mean.iloc[x]['+']
  var_b = contin_mean.iloc[y]['-']
  
  a = var_a.var(ddof=1)
  b = var_b.var(ddof=1)
  
  s = np.sqrt((var_a + var_b)/ 2)
  
  t_stat = (var_a.mean() - var_b.mean())/(s*np.sqrt(2/sample_size))
  
  degree_of_freedom = 2*sample_size - 2
  
  p_value = 1 - stats.t.cdf(t_stat, df=degree_of_freedom)
  
  x += 1
  y += 1
  
  print(t_stat)
 
  print(p_value)
  print('='*30)
  

17.67039900325078
0.0
26.90733419797209
0.0
45.08720937212393
0.0
-43.29357633264124
1.0
998.4540364370483
0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Degrees of freedom <= 0 for slice
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Degrees of freedom <= 0 for slice
  if sys.path[0] == '':


In [302]:
df.p.value_counts()

-    367
+    299
Name: p, dtype: int64

In [250]:
# p value looks wrong...
df_approved = df.loc[df['p'].isin(['+'])]

df_denied = df.loc[df['p'].isin(['-'])]


print(stats.ttest_ind(a = df_approved.o, b = df_denied.o))


print(stats.ttest_ind(a = df_approved.h, b = df_denied.h))


print(stats.ttest_ind(a = df_approved.c, b = df_denied.c))

Ttest_indResult(statistic=4.523020947003109, pvalue=7.221045410953856e-06)
Ttest_indResult(statistic=9.167860848961155, pvalue=5.960454400521998e-19)
Ttest_indResult(statistic=5.626602832074139, pvalue=2.7112481219534207e-08)


For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

In [284]:



credit_crosstab = pd.crosstab(df.p, [df.a, df.d, df.e, df.f, df.g, df.i, df.j, 
                                     df.l, df.m], 
            margins=True)


# hmmm, you could almost argue the unique combination of variables makes..a
# unique type of credit profile, and those profiles are more/less likely to be 
# approved


credit_list = credit_crosstab.sum().sort_values().tolist()


stats.chisquare(credit_list) # heh...

Power_divergenceResult(statistic=239552.04804804802, pvalue=0.0)

In [295]:
credit_crosstab_a = pd.crosstab(df.p, [df.a])
credit_crosstab_a

a,?,a,b
p,,,
+,3,95,201
-,9,108,250


In [296]:



# what value of a(for feature a, ugh, confusing) is expected to be approved in 
# the entire population

a_observed = 95 + 108

positive_total = 95 + 201

sample_total = 95 + 201 + 108 + 250

a_expected = a_observed *(positive_total/sample_total)
print("The expected value of approved a values in feature a is {}".format(
    a_expected))

chi_numer = ((a_observed - a_expected)**2)
chi_denom = a_expected

chi_square_a = chi_numer / chi_denom
print(chi_square_a)


stats.chisquare([95, 108], [201, 250])

The expected value of approved a values in feature a is 91.8776758409786
134.39794611124884


Power_divergenceResult(statistic=136.55649751243783, pvalue=1.507624211704122e-31)

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

Overall, the data was somewhat ambigious, and the t-tests on individiual variables, while interesting, did not yield much information, as the overall indication from the data is that a person is more likely to be denied for credit than approved. 

Where the data really starts to shine is when I crosstabbed the variables, and you can see "profiles" start to form for the data. If the correct categorical variables where met, then there was a direct increase in credit approval. Very fascinating.

